In [5]:
import os

In [6]:
abs_path = '/home/Akshath/PycharmProjects/Summarize4Me'
os.chdir(abs_path)
%pwd

'/home/Akshath/PycharmProjects/Summarize4Me'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    pretrained_model: Path
    num_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    evaluation_steps: int
    save_steps: float
    accumulation_steps: int

In [8]:
from Summarize4Me.constants import *
from Summarize4Me.utils.common import read_yaml, create_dirs

In [9]:
class ConfigManager:
    def __init__(
        self,
        config_fpath = CONFIG_FILE_PATH,
        params_fpath = PARAMS_FILE_PATH
    ):

        self.config = read_yaml(config_fpath)
        self.params = read_yaml(params_fpath)

        create_dirs([self.config.artifacts_root])

    def get_model_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_dirs([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            pretrained_model=config.pretrained_model,
            num_epochs=params.num_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            evaluation_steps=params.evaluation_strategy,
            save_steps=params.save_steps,
            accumulation_steps=params.accumulation_steps
        )

        return model_trainer_config

In [10]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

In [11]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.pretrained_model)
        pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.pretrained_model).to(device)
        seq2seq_collator = DataCollatorForSeq2Seq(tokenizer, model=pegasus)

        dataset_samsum = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=25,
            warmup_steps=500,
            per_device_train_batch_size=10,
            per_device_eval_batch_size=10,
            weight_decay=0.01,
            logging_steps=10,
            evaluation_strategy='steps',
            eval_steps=500,
            save_steps=1e9,
            gradient_accumulation_steps=40
        )

        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
        #     weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # )

        train = Trainer(
            model=pegasus,
            args=trainer_args,
            tokenizer=tokenizer,
            data_collator=seq2seq_collator,
            train_dataset=dataset_samsum["train"],
            eval_dataset=dataset_samsum["validation"]
        )

        train.train()

        pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))

        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

In [ ]:
try:
    config = ConfigManager()
    model_trainer_config = config.get_model_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-06-07 13:17:23,872: INFO: common: YAML File: config/config.yaml loaded]
[2023-06-07 13:17:23,875: INFO: common: YAML File: params.yaml loaded]
[2023-06-07 13:17:23,876: INFO: common: Created dir at: artifacts]
[2023-06-07 13:17:23,877: INFO: common: Created dir at: artifacts/model_trainer]


/home/Akshath/PycharmProjects/Summarize4Me/venv/lib/python3.11/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
